In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Flatten, Conv1D, MaxPool1D, Bidirectional

In [ ]:
def mlp(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        hidden_layers=[32, 16, 8],
        dropout=0.0):
    
    model = Sequential()
    
    # Flatten 2d input into 1d
    model.add(Flatten(input_shape=(input_shape[-2], input_shape[-1])))
    
    # Add Dense + Dropout for each hidden layer
    for hidden_units in hidden_layers:
        model.add(Dense(hidden_units))
        if dropout > 0:
            model.add(Dropout(dropout))
    
    # Output layer to predict {horizon} time steps ahead
    model.add(Dense(horizon))

    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
def cnn(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        conv_blocks=[[64, 7, 2], [128, 5, 2]],
        dense_layers=[],
        dense_dropout=0.0):
    
    # Split conv_blocks into three for easier use
    conv_layers, kernels, pool_sizes = [], [], []
    for layer, kern, pool in conv_blocks:
        conv_layers.append(layer)
        kernels.append(kern)
        pool_sizes.append(pool)

    model = Sequential()
    # First convolutional layer and pooling layer
    model.add(Conv1D(conv_layers[0], kernels[0], 
                     activation="relu", padding="same", 
                     input_shape=(input_shape[-2], input_shape[-1])))
    
    model.add(MaxPool1D(pool_size=pool_sizes[0]))
    
    # Rest of the conv + pool layers
    for chanels, kernel, pool_size in zip(conv_layers[1:], kernels[1:], pool_sizes[1:]):
        model.add(Conv1D(chanels, kernel, activation="relu", padding="same"))
        if pool_size:
            model.add(MaxPool1D(pool_size=pool_size))
    
    # Flatten input for dense layers
    model.add(Flatten())
    
    # Dense layers with dropout
    for hidden_units in dense_layers:
        model.add(Dense(hidden_units))
        if dense_dropout > 0:
            model.add(Dropout(dense_dropout))
    
    # Output layer
    model.add(Dense(horizon))

    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
def lstm(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        lstm_layers=[50],
        lstm_dropout=0.0,
        return_sequences=False,
        dense_layers=[],
        dense_dropout=0.0):
    
    model = Sequential()
    
    # Return sequences true if there are more LSTM layers after this one
    return_seq = return_sequences if len(lstm_layers) == 1 else True
    # First LSTM layer
    model.add(LSTM(lstm_layers[0], activation='relu', return_sequences=return_seq, 
                   dropout=lstm_dropout, input_shape=(input_shape[-2], input_shape[-1])))
    
    # Rest of the LSTM layers
    for i, u in enumerate(lstm_layers[1:]):
        return_seq = return_sequences if i == len(lstm_layers) - 2 else True
        model.add(LSTM(u, activation='relu', return_sequences=return_seq, dropout=lstm_dropout))
    
    if return_sequences:
        model.add(Flatten())
    
    # Add dense layers and dropout
    for units in dense_layers:
        model.add(Dense(units))
        if dense_dropout > 0:
            model.add(Dropout(dense_dropout))
    
    # Output dense layer
    model.add(Dense(horizon))
    
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
def bidirectional_lstm(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        lstm_layers=[50],
        lstm_dropout=0.0,
        return_sequences=False,
        dense_layers=[],
        dense_dropout=0.0):
    
    model = Sequential()
    
    # Return sequences true if there are more LSTM layers after this one
    return_seq = return_sequences if len(lstm_layers) == 1 else True
    # First LSTM layer
    model.add(Bidirectional(LSTM(lstm_layers[0], activation='relu', return_sequences=return_seq, 
                   dropout=lstm_dropout), input_shape=(input_shape[-2], input_shape[-1]))))
    
    # Rest of the LSTM layers
    for i, u in enumerate(lstm_layers[1:]):
        return_seq = return_sequences if i == len(lstm_layers) - 2 else True
        model.add(Bidirectional(LSTM(u, activation='relu', return_sequences=return_seq, dropout=lstm_dropout)))
    
    if return_sequences:
        model.add(Flatten())
    
    # Add dense layers and dropout
    for units in dense_layers:
        model.add(Dense(units))
        if dense_dropout > 0:
            model.add(Dropout(dense_dropout))
    
    # Output dense layer
    model.add(Dense(horizon))
    
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
def gru(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        gru_layers=[50],
        gru_dropout=0.0,
        return_sequences=False,
        dense_layers=[],
        dense_dropout=0.0):
    
    model = Sequential()
    
    # Return sequences true if there are more GRU layers after this one
    return_seq = return_sequences if len(gru_layers) == 1 else True
    # First GRU layer
    model.add(GRU(gru_layers[0], activation='relu', return_sequences=return_seq, 
                   dropout=gru_dropout, input_shape=(input_shape[-2], input_shape[-1])))
    
    # Rest of the GRU layers
    for i, u in enumerate(gru_layers[1:]):
        return_seq = return_sequences if i == len(gru_layers) - 2 else True
        model.add(GRU(u, activation='relu', return_sequences=return_seq, dropout=gru_dropout))
    
    if return_sequences:
        model.add(Flatten())
    
    # Add dense layers and dropout
    for units in dense_layers:
        model.add(Dense(units))
        if dense_dropout > 0:
            model.add(Dropout(dense_dropout))
    
    # Output dense layer
    model.add(Dense(horizon))
    
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
def encoder_decoder_lstm(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        encoder_layers=[100],
        encoder_dropout=0.0,
        decoder_layers=[100],
        decoder_dropout=0.0):
    
    model = Sequential()
    
    # Return sequences true if there are more encoder LSTM layers after this one
    return_seq = False if len(encoder_layers) == 1 else True
    # First encoder LSTM layer
    model.add(LSTM(lstm_layers[0], activation='relu', return_sequences=return_seq, 
                   dropout=lstm_dropout), input_shape=(input_shape[-2], input_shape[-1]))
    
    # Rest of the encoder LSTM layers
    for i, u in enumerate(encoder_layers[1:]):
        return_seq = False if i == len(encoder_layers) - 2 else True
        model.add(LSTM(u, activation='relu', return_sequences=return_seq, dropout=encoder_dropout))
    
    model.add(RepeatVector(horizon))
    
    # Decoder LSTM layers
    for i, u in enumerate(decoder_layers):
        model.add(LSTM(u, activation='relu', return_sequences=True, dropout=decoder_dropout))
    
    # Output dense layer
    model.add(TimeDistributed(Dense(1))
    
    model.compile(optimizer=optimizer, loss=loss)
    return model